In [2]:
import pandas as pd
import requests
import plotly.graph_objects as go

In [3]:
df_airports_columns = ['Airport ID', 'Name', 'City', 'Country', 'IATA', 'ICAO', 'Latitude', 'Longitude', 'Altitude', 'Timezone', 'Daylight Savings Time', 'Tz database time zone', 'Type', 'Source']

df_airports = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat', header=None, names=df_airports_columns)
df_airports.head()

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,Daylight Savings Time,Tz database time zone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


In [4]:
df_routes_columns = ['Airline', 'Airline ID', 'Source airport', 'Source airport ID', 'Destination airport', 'Destination airport ID', 'Codeshare', 'Stops', 'Equipment']

df_routes = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat', header=None, names=df_routes_columns)
df_routes.head()

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2


In [5]:
df_routes_with_distance = df_routes.merge(df_airports[['IATA', 'Latitude', 'Longitude']], how='left', left_on='Source airport', right_on='IATA').rename(columns={'IATA': 'Source IATA','Latitude': 'Source Latitude', 'Longitude': 'Source Longitude'})

df_routes_with_distance = df_routes_with_distance.merge(df_airports[['IATA', 'Latitude', 'Longitude']], how='left', left_on='Destination airport', right_on='IATA').rename(columns={'IATA': 'Destination IATA', 'Latitude': 'Destination Latitude', 'Longitude': 'Destination Longitude'})

df_routes_with_distance.drop(['Source IATA', 'Destination IATA'], axis=1, inplace=True)
df_routes_with_distance.dropna(subset=['Source Latitude', 'Source Longitude', 'Destination Latitude', 'Destination Longitude'], inplace=True)

df_routes_with_distance

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment,Source Latitude,Source Longitude,Destination Latitude,Destination Longitude
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2,43.449902,39.956600,55.606201,49.278702
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2,46.283298,48.006302,55.606201,49.278702
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2,46.283298,48.006302,44.225101,43.081902
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2,55.305801,61.503300,55.606201,49.278702
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2,55.305801,61.503300,55.012600,82.650703
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67658,ZL,4178,WYA,6334,ADL,3341,NaN,0,SF3,-33.058899,137.514008,-34.945000,138.531006
67659,ZM,19016,DME,4029,FRU,2912,NaN,0,734,55.408798,37.906300,43.061298,74.477600
67660,ZM,19016,FRU,2912,DME,4029,NaN,0,734,43.061298,74.477600,55.408798,37.906300
67661,ZM,19016,FRU,2912,OSS,2913,NaN,0,734,43.061298,74.477600,40.609001,72.793297


In [6]:
from math import sin, cos, sqrt, atan2, radians
from geopy.distance import geodesic

def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    earth_radius_km = 6371

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = earth_radius_km * c

    return distance

def calculate_geodesic_distance(lat1, lon1, lat2, lon2):
    point1 = (lat1, lon1)
    point2 = (lat2, lon2)

    distance = geodesic(point1, point2).km
    return distance

df_routes_with_distance['Great-circle distance'] = df_routes_with_distance.apply(
    lambda row: haversine(
        row['Source Latitude'], row['Source Longitude'], 
        row['Destination Latitude'], row['Destination Longitude']), 
    axis=1)

df_routes_with_distance['Geodesic Distance WGS-84'] = df_routes_with_distance.apply(
    lambda row: calculate_geodesic_distance(
        row['Source Latitude'], row['Source Longitude'], 
        row['Destination Latitude'], row['Destination Longitude']), 
    axis=1)

print(
    "Diferença em km entre diferentes métodos de cálculo da menor distância entre 2 pontos na superfície terrestre: ", 
    max(abs(df_routes_with_distance['Geodesic Distance WGS-84']-df_routes_with_distance['Great-circle distance']))
    )

df_routes_with_distance

Diferença em km entre diferentes métodos de cálculo da menor distância entre 2 pontos na superfície terrestre:  35.18291019776552


,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment,Source Latitude,Source Longitude,Destination Latitude,Destination Longitude,Great-circle distance,Geodesic Distance WGS-84
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2,43.449902,39.956600,55.606201,49.278702,1506.825604,1507.989680
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2,46.283298,48.006302,55.606201,49.278702,1040.438320,1040.943207
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2,46.283298,48.006302,44.225101,43.081902,448.164909,449.036664
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2,55.305801,61.503300,55.606201,49.278702,770.508500,773.126239
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2,55.305801,61.503300,55.012600,82.650703,1338.631467,1343.161122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67658,ZL,4178,WYA,6334,ADL,3341,NaN,0,SF3,-33.058899,137.514008,-34.945000,138.531006,229.720619,229.334786
67659,ZM,19016,DME,4029,FRU,2912,NaN,0,734,55.408798,37.906300,43.061298,74.477600,2942.819259,2949.906099
67660,ZM,19016,FRU,2912,DME,4029,NaN,0,734,43.061298,74.477600,55.408798,37.906300,2942.819259,2949.906099
67661,ZM,19016,FRU,2912,OSS,2913,NaN,0,734,43.061298,74.477600,40.609001,72.793297,306.295375,306.189294


In [7]:
data = requests.get('https://www.eurocontrol.int/performance/data/download/xls/Airport_Traffic.xlsx')

df_airport_traffic = pd.read_excel(data.content, sheet_name='DATA')
df_airport_traffic.head()

,YEAR,MONTH_NUM,MONTH_MON,FLT_DATE,APT_ICAO,APT_NAME,STATE_NAME,FLT_DEP_1,FLT_ARR_1,FLT_TOT_1,FLT_DEP_IFR_2,FLT_ARR_IFR_2,FLT_TOT_IFR_2,Pivot Label
0,2016,1,JAN,2016-01-01,EBAW,Antwerp,Belgium,4,3,7,NaN,NaN,NaN,Antwerp (EBAW)
1,2016,1,JAN,2016-01-01,EBBR,Brussels,Belgium,174,171,345,174.0,161.0,335.0,Brussels (EBBR)
2,2016,1,JAN,2016-01-01,EBCI,Charleroi,Belgium,45,47,92,45.0,45.0,90.0,Charleroi (EBCI)
3,2016,1,JAN,2016-01-01,EBLG,Liège,Belgium,6,7,13,NaN,NaN,NaN,Liège (EBLG)
4,2016,1,JAN,2016-01-01,EBOS,Ostend-Bruges,Belgium,7,7,14,NaN,NaN,NaN,Ostend-Bruges (EBOS)


In [8]:
# Adicionando uma coluna com o código IATA de cada aeroporto
df_airport_traffic = df_airport_traffic.merge(df_airports[['IATA', 'ICAO']], how='left', left_on='APT_ICAO', right_on='ICAO')

# Removendo a coluna do código ICAO dos aeroportos
df_airport_traffic.drop('ICAO',axis=1, inplace=True)

764508

In [24]:
# Criando uma cópia para preservar o dataframe original
df_airport_traffic_copy = df_airport_traffic.copy(deep=True)

# Numero de entradas
len(df_airport_traffic_copy.index)

# Removendo as entradas com código IATA em branco
df_airport_traffic_copy.dropna(subset=['IATA'], inplace=True)

# Removendo as linhas com código IATA inválido
df_airport_traffic_copy = df_airport_traffic_copy[df_airport_traffic_copy.IATA != '\\N']

# Numero de entradas após limpeza dos dados
len(df_airport_traffic_copy.index)

749154

In [25]:
# Removendo as colunas que não serão utilizadas
df_airport_traffic_copy = df_airport_traffic_copy.drop(['FLT_DATE','APT_ICAO','FLT_DEP_1','FLT_ARR_1','FLT_DEP_IFR_2','FLT_ARR_IFR_2','Pivot Label','FLT_TOT_IFR_2', 'MONTH_NUM', 'MONTH_MON', 'APT_NAME', 'STATE_NAME'],axis=1)


In [26]:
# Exibindo primeiro ano em que os dados foram colhidos por aeroporto
# Exemplo: O primeiro ano de monitoramento foi 2016 para 269 aeroportos
print(df_airport_traffic_copy.groupby('IATA')['YEAR'].min().value_counts())
print('\n')

# Exibindo o percentual do ano de início de monitoramento
print(df_airport_traffic_copy.groupby('IATA')['YEAR'].min().value_counts(normalize=True) * 100
)
print("\nPara que a VIS esteja correta, devemos comparar apenas os aeroportos cujos dados foram colhidos no mesmo intervalo de tempo.\nPor isso, vamos considerar o intervalo de 2018 a 2023 e desprezar os aeroportos cujo monitoramento iniciou após 2018.")

YEAR
2016    269
2018     47
2019      2
2021      2
Name: count, dtype: int64


YEAR
2016    84.0625
2018    14.6875
2019     0.6250
2021     0.6250
Name: proportion, dtype: float64

Para que a VIS esteja correta, devemos comparar apenas os aeroportos cujos dados foram colhidos no mesmo intervalo de tempo.
Por isso, vamos considerar o intervalo de 2018 a 2023 e desprezar os aeroportos cujo monitoramento iniciou após 2018.


In [27]:
# Mantendo apenas os dados a partir de 2018 e removendo as entradas que não possuem dados em 2018
df_airport_traffic_copy = df_airport_traffic_copy.query('YEAR >= 2018').groupby('IATA').filter(lambda x: x['YEAR'].min() == 2018)

# Somando as entradas e saídas em cada aeroporto no intervalo 2018-2023 
df_airport_traffic_grouped = df_airport_traffic_copy.groupby('IATA')[['FLT_TOT_1']].count().reset_index()

# Renomeando coluna do tráfego entre 2018-2023 para TOTAL_FLIGHTS
df_airport_traffic_grouped = df_airport_traffic_grouped.rename(columns={"FLT_TOT_1": "TOTAL_FLIGHTS"})
df_airport_traffic_grouped.head(5)


,IATA,TOTAL_FLIGHTS
0,ABC,1045
1,ABZ,1857
2,ACE,1857
3,ADB,1848
4,AES,1857


In [87]:
# Criando uma cópia do dataframe com as rotas e suas distâncias
df = df_routes_with_distance.copy(deep=True)

# Removendo as colunas que não serão utilizadas
df = df.drop(['Airline','Airline ID', 'Source airport ID', 'Destination airport ID','Codeshare', 'Stops', 'Equipment', 'Great-circle distance'],axis=1)

# Removendo entradas duplicadas (Diferentes companhias aereas não nos interessam)
df.drop_duplicates(inplace=True)

In [88]:
# Adicionando uma coluna para a quantidade total de voos no aeroporto de origem
df = df.merge(df_airport_traffic_grouped[['IATA', 'TOTAL_FLIGHTS']], how='left', left_on='Source airport', right_on='IATA').rename(columns={'IATA': 'Source IATA','TOTAL_FLIGHTS': 'Source Total Flights'})

# Removendo coluna que não será utilizada
df = df.drop(['Source IATA'], axis=1, inplace=False)

# Removendo os dados em branco
df = df.dropna()

In [89]:
df['Source Cost O'] = df['Source Total Flights']/df['Geodesic Distance WGS-84']
df.reset_index(drop=True, inplace=True)
df.columns = ['S_AIRPORT', 'D_AIRPORT', 'S_LAT', 'S_LON', 'D_LAT', 'D_LON', 'DISTANCE_WGS-84', 'S_TOTAL_TRAFFIC', 'S_COST']
df

,S_AIRPORT,D_AIRPORT,S_LAT,S_LON,D_LAT,D_LON,DISTANCE_WGS-84,S_TOTAL_TRAFFIC,S_COST
0,BDS,ZRH,40.657600,17.947001,47.464699,8.549170,1065.791972,1777.0,1.667305
1,BOD,ZRH,44.828300,-0.715556,47.464699,8.549170,772.820388,1856.0,2.401593
2,BRS,ZRH,51.382702,-2.719090,47.464699,8.549170,924.996473,1851.0,2.001089
3,GVA,LPA,46.238098,6.108950,27.931900,-15.386600,2771.225769,1857.0,0.670101
4,LCA,ZRH,34.875099,33.624901,47.464699,8.549170,2508.634058,1856.0,0.739845
...,...,...,...,...,...,...,...,...,...
10512,CDG,FAO,49.012798,2.550000,37.014400,-7.965910,1581.447209,1857.0,1.174241
10513,FAO,CDG,37.014400,-7.965910,49.012798,2.550000,1581.447209,1849.0,1.169182
10514,FNC,ORY,32.697899,-16.774500,48.723333,2.379444,2391.848817,1846.0,0.771788
10515,ORY,BKO,48.723333,2.379444,12.533500,-7.949940,4124.789966,1839.0,0.445841


In [90]:
df = df.merge(df_airports[['Name','City','Country','IATA']], how='left', left_on='S_AIRPORT', right_on='IATA').rename(
    columns={
        'Name': 'S_AIRPORT_NAME',
        'City': 'S_CITY',
        'Country': 'S_COUNTRY'
        }
    )
df.drop(['IATA'], axis=1, inplace=True)


df = df.merge(df_airports[['Name','City','Country','IATA']], how='left', left_on='D_AIRPORT', right_on='IATA').rename(
    columns={
        'Name': 'D_AIRPORT_NAME',
        'City': 'D_CITY',
        'Country': 'D_COUNTRY'
        }
    )
df.drop(['IATA'], axis=1, inplace=True)

In [91]:
df['S_COST_INVERSE'] = 1 / df['S_COST']
df

,S_AIRPORT,D_AIRPORT,S_LAT,S_LON,D_LAT,D_LON,DISTANCE_WGS-84,S_TOTAL_TRAFFIC,S_COST,S_AIRPORT_NAME,S_CITY,S_COUNTRY,D_AIRPORT_NAME,D_CITY,D_COUNTRY,S_COST_INVERSE
0,BDS,ZRH,40.657600,17.947001,47.464699,8.549170,1065.791972,1777.0,1.667305,Brindisi – Salento Airport,Brindisi,Italy,Zürich Airport,Zurich,Switzerland,0.599770
1,BOD,ZRH,44.828300,-0.715556,47.464699,8.549170,772.820388,1856.0,2.401593,Bordeaux-Mérignac Airport,Bordeaux,France,Zürich Airport,Zurich,Switzerland,0.416390
2,BRS,ZRH,51.382702,-2.719090,47.464699,8.549170,924.996473,1851.0,2.001089,Bristol Airport,Bristol,United Kingdom,Zürich Airport,Zurich,Switzerland,0.499728
3,GVA,LPA,46.238098,6.108950,27.931900,-15.386600,2771.225769,1857.0,0.670101,Geneva Cointrin International Airport,Geneva,Switzerland,Gran Canaria Airport,Gran Canaria,Spain,1.492313
4,LCA,ZRH,34.875099,33.624901,47.464699,8.549170,2508.634058,1856.0,0.739845,Larnaca International Airport,Larnaca,Cyprus,Zürich Airport,Zurich,Switzerland,1.351635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10512,CDG,FAO,49.012798,2.550000,37.014400,-7.965910,1581.447209,1857.0,1.174241,Charles de Gaulle International Airport,Paris,France,Faro Airport,Faro,Portugal,0.851614
10513,FAO,CDG,37.014400,-7.965910,49.012798,2.550000,1581.447209,1849.0,1.169182,Faro Airport,Faro,Portugal,Charles de Gaulle International Airport,Paris,France,0.855299
10514,FNC,ORY,32.697899,-16.774500,48.723333,2.379444,2391.848817,1846.0,0.771788,Madeira Airport,Funchal,Portugal,Paris-Orly Airport,Paris,France,1.295693
10515,ORY,BKO,48.723333,2.379444,12.533500,-7.949940,4124.789966,1839.0,0.445841,Paris-Orly Airport,Paris,France,Modibo Keita International Airport,Bamako,Mali,2.242953


In [ ]:
fig = go.Figure()

source_to_dest = zip(df['S_LAT'], df['D_LAT'],
                     df['S_LON'], df['D_LON'],
                     df['S_COST'])

## Loop thorugh each flight entry
for s_lat, d_lat, s_lon, d_lon, s_cost in source_to_dest:
    fig.add_trace(go.Scattergeo(
                        lat = [s_lat, d_lat],
                        lon = [s_lon, d_lon],
                        mode = 'lines',
                        line = dict(width = s_cost/50, color="red")
                        ))

fig.update_layout(title_text = 'Test',
                  height=700, width=900,
                  margin={"t":0,"b":0,"l":0, "r":0, "pad":0},
                  showlegend=False)

fig.show()

In [ ]:
fig = go.Figure()

source_to_dest = zip(df['S_LAT'], df['D_LAT'],
                     df['S_LON'], df['D_LON'],
                     df['S_COST'])

# Loop para cada entrada do dataframe, para adicionar uma linha entre a origem e o destino
for s_lat, d_lat, s_lon, d_lon, s_cost in source_to_dest:
    fig.add_trace(go.Scattergeo(
                        lat = [s_lat, d_lat],
                        lon = [s_lon, d_lon],
                        mode = 'lines',
                        line = dict(width = 0.1, color="red")
                        ))

# Criando os rótulos dos pontos do plot
cities = df['S_CITY'].values.tolist()+df['D_CITY'].values.tolist()
countries = df['S_COUNTRY'].values.tolist()+df['D_COUNTRY'].values.tolist()
scatter_hover_data = [country + " : "+ city for city, country in zip(cities, countries)]

fig.add_trace(
    go.Scattergeo(
                lon = df['S_LON'].values.tolist()+df['D_LON'].values.tolist(),
                lat = df['S_LAT'].values.tolist()+df['D_LAT'].values.tolist(),
                hoverinfo = 'text',
                text = scatter_hover_data,
                mode = 'markers',
                marker = dict(
                    size = df['S_COST'].values, 
                    #color = 'blue', 
                    colorscale = 'Blues',
                    cmin = 0,
                    color = df['S_COST'],
                    cmax = df['S_COST'].max(),
                    colorbar_title="Fator custo entre as rotas internacionais dos aeroportos",
                    opacity=0.1
                    ))
    )

# Alterando o layout do gráfico
fig.update_layout(title_text="Connection Map Depicting International Flight Routes",
                  height=700, width=900,
                  margin={"t":0,"b":0,"l":0, "r":0, "pad":0},
                  showlegend=True,
                  geo= dict(showland = True, landcolor = 'white', countrycolor = 'grey', bgcolor="lightgrey"))

fig.show()
